# Floating Lantern Feasibility

This document outlines calculations for a buoyant lamp. The primary buoyancy should be provided by a lifting gas (H2 or He). The buoyant lamp is inspired by floating lamps that appear in many sci-fi stories. 

## Requirements/Goals

* It can be built and operated without access to special materials
* It should be quiet (i.e. not a drone)
* It's designed to work at night or indoors (e.g. no solar heating load, low luminance)
* It should be able to remain stationary in low velocity wind (less than 1m/s, TBD)
* It can require physical balast (e.g. water) to accomodate coarse changes in ambient air pressure (e.g. by elevation); it may require a lower power lifting propeller to establish sufficient buoyancy

## Lifting Body

### Material & Physical Properties

#### References
1. [Lifting gas/Hydrogen](https://en.wikipedia.org/wiki/Lifting_gas#Hydrogen)
2. [Hydrogen](https://en.wikipedia.org/wiki/Hydrogen)
3. [Mylar film](https://dupontteijinfilms.us/wp-content/uploads/2017/01/Mylar_Physical_Properties.pdf)

#### Notes
* Working units are "cgs" for convenience of scale.
* "Mylar" film (Al coated polyester?) density from kitchen scale measurement of a dollar store emergency blanket (e.g. like [this](https://www.mec.ca/en/product/0199-034/emergency-blanket?colour=NO_COLOUR) but half the price).
* Tissue paper density also from the kitchen scale, product from the dollar store (e.g. for gift wrapping).

In [1]:
import numpy as np
import scipy.constants as cte
import tabulate
from IPython.display import HTML, display

rho_air = 1.29          # g/L
rho_h2 = 0.09           # g/L
rho_he = 0.18           # g/L
rhos_mylar = 1.634e-3   # g/cm2
rhos_tissue = 1.414e-3  # g/cm2
rho_h2o = 1000          # g/L
rho_ss304 = 8           # g/cm3

mat_props = [['Dry air', rho_air, 'g/L'], 
             ['H2', rho_h2, 'g/L'], 
             ['He', rho_he, 'g/L'], 
             ['H20', rho_h2o, 'g/L'],
             ['Mylar foil', rhos_mylar, 'g/cm2'],
             ['Tissue paper', rhos_tissue, 'g/cm2']]
            
display(HTML(tabulate.tabulate(mat_props, headers=['Material', 'Density (@STP)', 'Units'], floatfmt=".4f", tablefmt='html')))


Material,Density (@STP),Units
Dry air,1.2900,g/L
H2,0.0900,g/L
He,0.1800,g/L
H20,1000.0000,g/L
Mylar foil,0.0016,g/cm2
Tissue paper,0.0014,g/cm2


### Physical Equations

#### Buoyant Force (Archimedes Principle)
The buoyancy force $F_B = (\rho_{f} - \rho{b})gV$ where $V$ is the displaced volume of the fluid $f$. Equivalently,  
$$\begin{align*}
F_B = mg &= (\rho_{f} - \rho_{b})gV \\
\to m &= (\rho_{f} - \rho_{b})V \\
\to V &= m(\rho_{f} - \rho_{b})^{-1}
\end{align*}$$
which implies that a volume of gas $V$ is required to lift mass $m$

#### Static Thrust
The ideal static thrust of a propeller in relation to the applied power is
$$
T_{0} = \left( 2\rho S P^2 \right)^{1/3}\, \mathrm{[N]}
$$
where $\rho$ is the fluid (air) density, $S$ the area swept out by the propeller ($S=\pi r_{prop}^2$), and $P$ the applied power from the motor. This formula assumes that the inlet flow velocity is zero and that the propeller is perfectly efficient (scale by some $\eta$ to correct this).

#### Drag
The drag of a body depends on its "reference area" $A_{ref}$ (for simple convex solids, this is the projected cross-section orthogonal to the flow direction) and the coefficient of drag $c_d$:
$$
F_d = \frac{1}{2}\rho u^2 A_{ref} c_d\, \mathrm{[N]}
$$
where $\rho$ is the fluid (air) density and $u$ is the fluid velocity. 

Some assorted drag coefficients can be found on [Wikipedia](https://en.wikipedia.org/wiki/Drag_coefficient). For this body, assume $A_ref$ is the rectangular projection of the upright cylinder ($A_{ref} = h\times d$) and $c_d = 1.17$ (cylinder). 

### Balloon and Lantern Characteristics

In this section, we tabulate the characteristics of a model lantern. The construction is assumed to consist of a cylindrical Mylar balloon as a core inside of a tissue paper lantern.

* The cylinder shape is selected since it's reasonably easy to construct, offers the ability to trade off aspect ratio, and any shape with sharp corners (e.g. a cube) would probably look like a cylinder if made of flimsy plastic and inflated.
* The offset of the lantern paper from the core balloon is a design parameter

#### Design Parameters
* Designed lifting force (mass g)
* Cylinder radius (cm)
* Lantern shell offset (cm)
* Max wind speed (cm/s)

In [3]:
cm3_per_L = 1000

r_cyl = 20                          # cm
lift_mass = np.linspace(50,100,6)   # g
shell_offset = 2                    # cm
u_wind_max = 100                    # cm/s

class LanternModel:
    prop_keys = ['Volume (L)', 'H (cm)', 'S (cm2)', 'So (cm2)', 'Mylar mass (g)', 
                 'Tissue mass (g)', 'Payload cap. (g)', 'Fd (N)']

    def __init__(self, m_lift, gas_type, r_cyl, t_offset, u_wind):
        self.c_d_cyl = 1.17

        self.m_lift = m_lift
        self.r_cyl = r_cyl
        self.t_shell_offset = t_offset
        self.u_wind = u_wind
        
        d_rho = rho_air     # g/L
        if gas_type == "H2":
            d_rho -= rho_h2
        elif gas_type == "He":
            d_rho -= rho_he
        else:
            raise ValueError("Invalid gas type {}".format(gas_type))
        
        v_h2 = m_lift/d_rho                                     # L
        h_cyl = v_h2*cm3_per_L / (np.pi * r_cyl**2)             # cm
        S_cyl = h_cyl * (2 * np.pi * r_cyl) + 2*np.pi*r_cyl**2  # cm2
        ho = h_cyl + 2*t_offset                 
        ro = r_cyl + t_offset
        So_cyl = ho * (2 * np.pi * ro) + 2*np.pi*ro**2          # cm2
        m_mylar = rhos_mylar * S_cyl
        m_tissue = rhos_tissue * So_cyl

        self.prop_table = {}
        self.prop_table[self.prop_keys[0]] = v_h2
        self.prop_table[self.prop_keys[1]] = h_cyl
        self.prop_table[self.prop_keys[2]] = S_cyl
        self.prop_table[self.prop_keys[3]] = So_cyl
        self.prop_table[self.prop_keys[4]] = m_mylar
        self.prop_table[self.prop_keys[5]] = m_tissue
        self.prop_table[self.prop_keys[6]] = self.payload_capacity()
        self.prop_table[self.prop_keys[7]] = self.drag()

    def __getitem__(self, key):
        return self.prop_table[key]

    def __setitem__(self, key, item):
        if key not in self.prop_keys:
            raise KeyError("Key not in allowed keys")

        self.prop_table[key] = item
        self._invalidate_prop_table()

    def drag(self):
        ho = self.prop_table[self.prop_keys[1]] + 2*self.t_shell_offset                 
        ro = self.r_cyl + self.t_shell_offset
        Aref = ho * (2*ro)
        return 0.5 * (rho_air / cm3_per_L) * self.u_wind**2 * Aref * self.c_d_cyl / 1000 / 100 # N

    def payload_capacity(self):
        return self.m_lift - self.prop_table[self.prop_keys[4]] - self.prop_table[self.prop_keys[5]]

    def _invalidate_prop_table(self):
        self.prop_table[self.prop_keys[6]] = self.payload_capacity()
        self.prop_table[self.prop_keys[7]] = self.drag()


lanterns = [LanternModel(ml, gas_type="H2", r_cyl=r_cyl, t_offset=shell_offset, u_wind=u_wind_max) for ml in lift_mass]
data_table = [[k] + [l[k] for l in lanterns] for k in LanternModel.prop_keys]
print("H2 Lantern Models")
display(HTML(tabulate.tabulate(data_table, headers=[""] + ["{}g".format(m) for m in lift_mass], floatfmt=".3f", tablefmt='html')))

lanterns_he = [LanternModel(ml, gas_type="He", r_cyl=r_cyl, t_offset=shell_offset, u_wind=u_wind_max) for ml in lift_mass]
data_table_he = [[k] + [l[k] for l in lanterns_he] for k in LanternModel.prop_keys]
print("He Lantern Models")
display(HTML(tabulate.tabulate(data_table_he, headers=[""] + ["{}g".format(m) for m in lift_mass], floatfmt=".3f", tablefmt='html')))

print("d_cyl={}cm | max(u_wind)={}m/s --> h_golden={:.2f}cm".format(2*r_cyl, u_wind_max/100, 1.681*2*r_cyl))



H2 Lantern Models


,50.0g,60.0g,70.0g,80.0g,90.0g,100.0g
Volume (L),41.667,50.000,58.333,66.667,75.000,83.333
H (cm),33.157,39.789,46.420,53.052,59.683,66.315
S (cm2),6679.941,7513.274,8346.607,9179.941,10013.274,10846.607
So (cm2),8177.315,9093.982,10010.649,10927.315,11843.982,12760.649
Mylar mass (g),10.915,12.277,13.638,15.000,16.362,17.723
Tissue mass (g),11.563,12.859,14.155,15.451,16.747,18.044
Payload cap. (g),27.522,34.864,42.207,49.549,56.891,64.233
Fd (N),0.123,0.145,0.167,0.189,0.211,0.233


He Lantern Models


,50.0g,60.0g,70.0g,80.0g,90.0g,100.0g
Volume (L),45.045,54.054,63.063,72.072,81.081,90.090
H (cm),35.846,43.015,50.184,57.353,64.522,71.691
S (cm2),7017.779,7918.680,8819.580,9720.481,10621.382,11522.283
So (cm2),8548.937,9539.928,10530.919,11521.910,12512.901,13503.892
Mylar mass (g),11.467,12.939,14.411,15.883,17.355,18.827
Tissue mass (g),12.088,13.489,14.891,16.292,17.693,19.095
Payload cap. (g),26.445,33.571,40.698,47.825,54.951,62.078
Fd (N),0.132,0.156,0.180,0.204,0.228,0.251


d_cyl=40cm | max(u_wind)=1.0m/s --> h_golden=67.24cm


### Tissue Paper Frame

A model paper lantern style is the cylindrical Japanese paper lantern. These look like they're constructed by layering strips of tissue or rice paper over a coiled wire frame. Hard stainless steel wire (e.g. 304) comes in various diameters. 

In [4]:
d_wire = 0.05                   # cm
A_wire = 0.25*np.pi*d_wire**2   # cm2
rhol_wire = rho_ss304*A_wire    # g/cm
N_rings = 5
L_rings = N_rings * np.pi * 44  # cm

print("{:.2f}mm diameter SS wire: {} g/m".format(d_wire*10, rhol_wire*100))
print("{} rings with 44cm diameter: {}m ({}g)".format(N_rings, L_rings/100, L_rings*rhol_wire))

0.50mm diameter SS wire: 1.5707963267948968 g/m
5 rings with 44cm diameter: 6.911503837897545m (10.856564841198296g)


### Conclusions/Notes

* For 100g gross lifting capacity (90g with He), a 40cm diameter cylindrical balloon gives a nice ratio to H
* At 50% efficiency, two 75mm props will require about 3W input power each to overcome wind-induced drag
    * Given a 3.7V 800mAh LiPo supply, current draw for 6W is 1.6A, equivalent to 30 mins run time
    * May need to reduce wind limit (full power would also be noisy)
* Tissue paper & frame can easily add to 30g+, more than a third of the mass budget for the 100g lift. It also needs a frame to support it -- might not be feasible.

In [6]:
def static_thrust_power(T_out, d_prop, eta=1):
    # T = cbrt(2*rho*S*P^2)
    S = 0.25*np.pi*d_prop**2
    return np.sqrt((T_out/eta)**3 / 2 / rho_air / S)

Fd = 0.25       # N
d_prop = 0.075  # m (3in = 76mm)
eta = 0.5
print("{:.2f}W at {}% efficiency for {}N ({:.1f} g) thrust with a {}mm dia. prop".format(
    static_thrust_power(Fd, d_prop, eta),
    eta*100,
    Fd,
    Fd/cte.g*1000,
    d_prop*1000))



3.31W at 50.0% efficiency for 0.25N (25.5 g) thrust with a 75.0mm dia. prop


## Hydrogen Production

### Electrolysis
Generating hydrogen gas is planned from water with NaOH or KOH added as an electrolyte (H2 is collected at the cathode). 

#### Material Properties
* Hydrogen $H_2$ molar mass $M_{H_2} = 2.016 \mathrm{g/mol}$
* Water $H_2 O$ molar mass $M_{H_2 O} = 18.016 \mathrm{g/mol}$

#### Reaction
* cathode (reduction): $2H_2 O(l) + 2e^- \to H_2(g) + 2OH^-(aq)$
* anode (oxidation): $2OH^-(aq) \to 1/2 O_2(g) + H_2 O(l) + 2e^-$

#### Efficiency
* A liter of $H_2$ at STP has mass $m_{H_2} = \rho_{H_2}\times 1\mathrm{L}$
* There are $m_{H_2}/M_{H_2}$ mols of $H_2$ per liter
* Each mol of $H_2$ production requires 2 mols of electrons and 2 mols of $H_2 O$



In [6]:

mmol_h2 = 2.016     # g/mol
mmol_h2o = 18.016   # g/mol

mh2_1L = rho_h2*1    # g
mols_h2_1L = mh2_1L/mmol_h2

mols_h2o = 2*mols_h2_1L
mh2o_1L = mmol_h2o * mols_h2o   # water mass to produce 1L H2
vh2o_1L = mh2o_1L/rho_h2o       # volume of water to produce 1L H2

#print("mols_h2o={:.3f}, mh2o_1L={:.3f}".format(mols_h2o, mh2o_1L))
print('{:.3f}mL H2O required to produce 1L H2'.format(vh2o_1L*1000))

mols_e = 2*mols_h2_1L
mols_per_C = 1.036e-5
I_dc = np.linspace(1,10,10)     # current supplied in A
mols_e_per_sec = I_dc*mols_per_C

T_1L = mols_e / mols_e_per_sec

electrolysis_table = np.zeros((len(I_dc), 3))
electrolysis_table[:,0] = I_dc
electrolysis_table[:,1] = T_1L/60
electrolysis_table[:,2] = 90*T_1L/60/60
display(HTML(tabulate.tabulate(electrolysis_table, headers=["I_DC (A)", "T_1L (min)", "T_90L (hrs)"], floatfmt=".3f", tablefmt='html')))


1.609mL H2O required to produce 1L H2


I_DC (A),T_1L (min),T_90L (hrs)
1.000,143.639,215.458
2.000,71.819,107.729
3.000,47.880,71.819
4.000,35.910,53.864
5.000,28.728,43.092
6.000,23.940,35.910
7.000,20.520,30.780
8.000,17.955,26.932
9.000,15.960,23.940
10.000,14.364,21.546


#### Conclusions/Notes
* Production is slow (24 hrs/80L balloon)
* Water required is small (100mL), so container size to hold >10x volume should be around 2L
* With good conductivity in solution, possible to reach 10A at approximately 4V (according to Nighthawkinlight): 40W

### Aluminium and HCl or NaOH Solution

#### References
* [NighthawkInLight](https://www.youtube.com/watch?v=f-HkVlq1kcQ).
* [Chemistry site](https://www.chemistryscl.com/reactions/aluminum+NaOH%20reaction/index.php)

#### Materials
* HCL (muriatic acid) and/or NaOH (lye)
* Water
* Aluminium foil

#### Reaction
* in muriatic acid: $2Al + 6HCl(aq) \to 2AlCl_3(aq) + 3H_2(g)$
* in aqueous $NaOH$: $2Al + 2NaOH(aq) + 2H_2O \to 2NaAlO_2(aq) + 3H_2(g)$


In [7]:
mmol_naoh = 40      # g/mol
mmol_al = 26.98     # g/mol
# mols_h2_1L from previous
target_vol = 10     #L
mass_naoh = target_vol*mols_h2_1L*(2/3)*mmol_naoh
mass_al = target_vol*mols_h2_1L*(2/3)*mmol_al

print('{}L of H2 requires {:.2f}g of NaOH and {:.2f}g Al'.format(target_vol, mass_naoh, mass_al))

10L of H2 requires 11.90g of NaOH and 8.03g Al


## Control System
### Overivew
#### Inputs
* Barometric altitude (AGL)
* Position (relative to origin) -- proposed inside-out camera tracking
* Attitude (6DOF+magnetic compass)

#### Outputs
* Independent left/right motor velocity (PWM duty), forward/backwards
* Motor tilt (+/-90deg range)

### Altitude
* Hold based on pressure (BMP280 sensor w/ averaging)
* May be possible to do with second camera (binocular) - possible with single camera and feature identfication given calibration?

### Position
* Inside-out tracking with digial image correlation
    * Need to calibrate pixel resolution (e.g. cm/pix) or possible with closed loop control?
    * Need subpixel resolution (assume some windowing function)?

### Attitude
* Utility for thrust vectoring?
* Magnetic orientation for image rotation (position tracking)
